In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import os, csv, time, copy
import numpy as np
from sklearn.externals import joblib
from sklearn.metrics import r2_score
from sklearn.preprocessing import normalize, MinMaxScaler

from sklearn.metrics.pairwise import linear_kernel
from multichannel.multichannel_model import MultiChannelModel, multichannel_KFoldCV
from multichannel.theano_kernels import theano_rbf as rbf_kernel, theano_chi2 as chi2_kernel

from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from multichannel.weighted_ridge import WeightedRidge

from hyperopt import hp, fmin, tpe, Trials

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN not available)


In [2]:
#helper save function
save_root = '/home/luke/projects/video_regression/code/ml_pl/hyperopt_results'
def save_trials(trials, name):
    ext = '.pkl'
    save_path = os.path.join(save_root, name + ext)
    print save_path
    joblib.dump(trials, save_path)
def load_trials(name):
    ext = '.pkl'
    save_path = os.path.join(save_root, name + ext)
    print save_path
    return joblib.load(save_path)

In [3]:
#load the data
dataset_root = '/home/luke/projects/THE_dataset' #directory where features/labels kept
train_path = os.path.join(dataset_root, 'train_set_wc3d.pkl')
test_path = os.path.join(dataset_root, 'test_set_wc3d.pkl')
X_train, y_train = joblib.load(train_path)
X_test, y_test = joblib.load(test_path)

In [4]:
#set up search space for linear models
svr_lin_space = {'C': hp.lognormal('svr_C', 0, 1)}
ridge_lin_space = {'alpha': hp.lognormal('ridge_alpha', 0, 1)}

In [5]:
#linear objectives
def svr_lin_objective(input_dict):
    from sklearn.svm import SVR
    from sklearn.metrics.pairwise import linear_kernel
    from multichannel.multichannel_model import MultiChannelModel, multichannel_KFoldCV
    from multichannel.theano_kernels import theano_rbf as rbf_kernel, theano_chi2 as chi2_kernel
    import csv, os, time, copy
    import numpy as np
    
    #load the dataset
    dataset_root = '/home/luke/projects/THE_dataset' #directory where features/labels kept
    train_path = os.path.join(dataset_root, 'train_set_wc3d.pkl')
    test_path = os.path.join(dataset_root, 'test_set_wc3d.pkl')
    X_train, y_train = joblib.load(train_path)
    X_test, y_test = joblib.load(test_path)
    
    kpt = {'kernel_func': linear_kernel, 'param_dict': {}}
    C = input_dict['C']
    model = SVR(kernel='precomputed', C=C)
    mcm = MultiChannelModel(num_channels=6, model=model, kernel_param_tuple=kpt)
    scores = multichannel_KFoldCV(mcm, X_train, y_train, n_folds=3, verbose=False)
    loss = 1-np.mean(scores)
    eval_time = time.time()
    
    #logging
    with open('svr_lin_log.csv','a') as f:
        fc = csv.writer(f)
        row = [loss, eval_time, C] + gammas
        row = [str(a) for a in row]
        fc.writerow(row)
    return {'loss': loss, 'eval_time': eval_time}

def ridge_lin_objective(input_dict):
    kpt = {'kernel_func': linear_kernel, 'param_dict': {}}
    alpha = input_dict['alpha']
    model = Ridge(alpha=alpha)
    mcm = MultiChannelModel(num_channels=6, model=model, kernel_param_tuple=kpt)
    scores = multichannel_KFoldCV(mcm, X_train, y_train, n_folds=3, verbose=False)
    return {'loss': 1-np.mean(scores), 'eval_time': time.time()}

In [6]:
#nonlinear search spaces
svr_nonlin_space = {'C': hp.lognormal('svr_C', 0, 1.5),
                 'traj_gamma': hp.lognormal('traj_gamma', 0, 1),
                 'hog_gamma': hp.lognormal('hog_gamma', 0, 1),
                 'hof_gamma': hp.lognormal('hof_gamma', 0, 1),
                 'mbhx_gamma': hp.lognormal('mbhx_gamma', 0, 1),
                 'mbhy_gamma': hp.lognormal('mbhy_gamma', 0, 1),
                 'c3d_gamma': hp.lognormal('c3d_gamma', 0, 1), 
                 'traj_cw': hp.uniform('traj_cw', 0, 1),
                 'hog_cw': hp.uniform('hog_cw', 0, 1),
                 'hof_cw': hp.uniform('hof_cw', 0, 1),
                 'mbhx_cw': hp.uniform('mbhx_cw', 0, 1),
                 'mbhy_cw': hp.uniform('mbhy_cw', 0, 1),
                 'c3d_cw': hp.uniform('c3d_cw', 0, 1)}

ridge_nonlin_space = copy.copy(svr_nonlin_space)
ridge_nonlin_space.pop('C', None)
ridge_nonlin_space['alpha'] = hp.lognormal('ridge_alpha', 0, 1.5)

In [7]:
#nonlinear objectives
#helper function for creating the kpt
def create_kpt(num_channels, gammas):
    kernel_param_list = []
    for channel in xrange(num_channels):
        if channel < 4:
            kdict = {'kernel_func': chi2_kernel, 'param_dict': {'gamma': gammas[channel]}}
        elif channel == 5:
            kdict = {'kernel_func': rbf_kernel, 'param_dict': {'gamma': gammas[channel]}}
        kernel_param_list.append(kdict)
    kernel_param_tuple = tuple(kernel_param_list)
    return kernel_param_tuple

def svr_nonlin_objective(x):
    gammas = [x['traj_gamma'], x['hog_gamma'], x['hof_gamma'],
              x['mbhx_gamma'], x['mbhy_gamma'], x['c3d_gamma']]
    kpt = create_kpt(6, gammas)
    C = x['C']
    model = SVR(kernel='precomputed', C=C)
    mcm = MultiChannelModel(num_channels=6, model=model, kernel_param_tuple=kpt)
    scores = multichannel_KFoldCV(mcm, X_train, y_train, n_folds=3, verbose=False)
    loss = 1-np.mean(scores)
    eval_time = time.time()
    
    #logging
    with open('svr_nonlin_log.csv','a') as f:
        fc = csv.writer(f)
        row = [loss, eval_time, C] + gammas
        row = [str(a) for a in row]
        fc.writerow(row)
    return {'loss': loss, 'eval_time': eval_time}

def ridge_nonlin_objective(x):
    gammas = [x['traj_gamma'], x['hog_gamma'], x['hof_gamma'],
              x['mbhx_gamma'], x['mbhy_gamma'], x['c3d_gamma']]
    kpt = create_kpt(6, gammas)
    alpha = x['alpha']
    model = Ridge(alpha=alpha)
    mcm = MultiChannelModel(num_channels=6, model=model, kernel_param_tuple=kpt)
    scores = multichannel_KFoldCV(mcm, X_train, y_train, n_folds=3, verbose=False)
    loss = 1-np.mean(scores)
    eval_time = time.time()
    
    #logging
    with open('ridge_nonlin_log.csv','a') as f:
        fc = csv.writer(f)
        row = [loss, eval_time, alpha] + gammas
        row = [str(a) for a in row]
        fc.writerow(row)
    return {'loss': loss, 'eval_time': eval_time}

In [ ]:
def run_exp(objective, space, name, max_evals=10):
    #set up trials and run experiment
    trials = Trials()
    best = fmin(objective,
        space=space,
        algo=tpe.suggest,
        max_evals=max_evals,
        trials=trials)
    #save the trials structure
    save_trials(trials, name)
    return trials, best

In [ ]:
ridge_nonlin_trials, best = run_exp(objective=ridge_nonlin_objective, 
                                  space=ridge_nonlin_space, name='ridge_nonlin', max_evals=1500)

In [2]:
ridge_nonlin_trials

NameError: name 'ridge_nonlin_trials' is not defined

In [98]:
svr_nonlin_trials = load_trials('svr_nonlin')

/home/luke/projects/video_regression/code/ml_pl/hyperopt_results/svr_nonlin.pkl


In [115]:
svr_nonlin_trials.results[-1]['eval_time']-svr_nonlin_trials.results[0]['eval_time']

332.2247200012207

In [ ]:
ridge_lin_trials = run_exp(objective=svr_lin_objective, 
                           space=svr_lin_space, name='svr_lin', max_evals=10)